In [1]:
from keras.models import load_model
import tensorflowjs as tfjs
import cv2
import numpy as np
model = load_model('Face_Model/Models/{NAME}.model')
# tfjs.converters.save_keras_model(model, 'models')


In [2]:

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}
while(True):

    ret,img=source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, (labels_dict[label]), (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

In [3]:
def prepare(filepath):
    img_size = 100 
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)/255  
    img_resize = cv2.resize(img, (img_size, img_size))  
    return img_resize.reshape(-1, img_size, img_size, 1)

In [6]:
import csv
path = "Face_Mask/Test"
with open('Face_Mask_Result.csv', mode='w') as features:
  features_writer = csv.writer(features, delimiter=',')
  features_writer.writerow(['Features extracted from Pictures',"Result"])
  for folder in (os.listdir(path)):
    fp = os.path.join(path,folder)
    for eachImage in os.listdir(fp):
      imagePath = os.path.join(fp,eachImage)
      prediction = model.predict(prepare(imagePath))
      CATEGORIES = ["without_mask","with_mask"]
      pred_class = CATEGORIES[np.argmax(prediction)] 
      features_writer.writerow([imagePath,pred_class])